In [59]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import math
import json
import numpy as np
from matplotlib.ticker import MaxNLocator
from os.path import exists
import pickle
from scipy.stats import sem
import statistics
import scipy
from sklearn.metrics import mean_squared_error
from kneed import KneeLocator
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
##### Save self finding steps in contingency game for each level #####

## Get no-movement action count of agent for each level
def get_self_finding_steps(game_type):
    files = glob.glob("../data/" + game_type + "/data_sf/*.json")

    lvl_count = 53 if 'change_agent_game' in game_type else 100

    if 'change_agent_game' in game_type:
        all_levels = {'steps': [], 'self_finding_steps': [], 'prop_selected_correctly': [], 'participant': [], 'level': []}
    else:
        all_levels = {'steps': [], 'self_finding_steps': [], 'participant': [], 'level': []}
    for i, file in enumerate(files):  # Get the data for each participant in that level
        print(file)
        data = json.load(open(file))

        for level in range(0, lvl_count):  # Data for each level
            selected_agents = data.get("selectedAgents")

            # Correctly clicked agents in a level
            correct_sf_level = [d for d in selected_agents if d['level'] == level and d['real_self'] == d['clicked_self']]

            if 'change_agent_game' in game_type:
                # Clicked agents in a level (regardless of whether they are correct or not)
                level_clicks = [d for d in selected_agents if d['level'] == level]
                if len(level_clicks) >= 1:
                    avg_sf_steps = []
                    clicks_clean = copy.deepcopy(level_clicks)

                    last_click_emb = None
                    last_click = None
                    for click in level_clicks:
                        if len(clicks_clean) == 1:
                            break

                        if last_click_emb is None:
                            last_click_emb = math.floor(click['step'] / 7)
                        else:
                            if math.floor(click['step'] / 7) == last_click_emb:
                                # Remove the one before (one with lower step)
                                clicks_clean.remove(last_click)
                            last_click_emb = math.floor(click['step'] / 7)
                        
                        last_click = click

                    
                    avg_sf_steps = [c['step'] % 7 if c['real_self'] == c['clicked_self'] else 7 for c in clicks_clean]
                    all_levels['self_finding_steps'].append(statistics.mean(avg_sf_steps) if len(avg_sf_steps) > 0 else None)
                    all_levels['prop_selected_correctly'].append(statistics.mean([d['real_self'] == d['clicked_self'] for d in clicks_clean]) if len(clicks_clean) > 0 else None)
                    all_levels['steps'].append(data['data']['steps'][level])
                else:
                    all_levels['self_finding_steps'].append(None)
                    all_levels['prop_selected_correctly'].append(None)
                    all_levels['steps'].append(data['data']['steps'][level])
            else:
                if len(correct_sf_level) == 1:  # Clicked the correct agent in a level
                    all_levels['steps'].append(data['data']['steps'][level])
                    all_levels['self_finding_steps'].append(correct_sf_level[0]['step'])
                else:  # Did not click the correct agent in a level
                    all_levels['steps'].append(data['data']['steps'][level])
                    all_levels['self_finding_steps'].append(None)
            
            all_levels['participant'].append(i)
            all_levels['level'].append(level)

    df = pd.DataFrame(all_levels)
    df.to_csv('../stats/self_orienting_{}.csv'.format(game_type))

In [61]:
for game in ["change_agent_game"]: #"contingency_game", "contingency_game_shuffled_1", 
    get_self_finding_steps(game)

../data/change_agent_game/data_sf/A1SN7EOZUW5VP3_2023-04-28=14:36:47.json
../data/change_agent_game/data_sf/A10HW8JXM17XLD_2023-04-28=15:13:48.json
../data/change_agent_game/data_sf/A2Y7L7R5UTQ5R2_2023-04-28=16:14:59.json
../data/change_agent_game/data_sf/A2620VELR4LMLU_2023-04-28=17:01:01.json
../data/change_agent_game/data_sf/A1GGBLQXMJP6YU_2023-04-28=15:40:08.json
../data/change_agent_game/data_sf/A334VWE4QGAZFR_2023-04-28=18:22:34.json
../data/change_agent_game/data_sf/A3AHD1NV5XCOF_2023-04-28=17:24:44.json
../data/change_agent_game/data_sf/A1OFFNJXVHVI64_2023-04-28=19:28:59.json
../data/change_agent_game/data_sf/A3O3NJQ5AUXUVV_2023-04-28=18:21:59.json
../data/change_agent_game/data_sf/AIHHRINW9IN1I_2023-04-28=15:13:49.json
../data/change_agent_game/data_sf/A1J7GK3Z74R4HU_2023-04-28=16:40:08.json
../data/change_agent_game/data_sf/A13T1T1VPOCMD3_2023-04-28=18:02:03.json
../data/change_agent_game/data_sf/A80WPJSYIU9LT_2023-04-28=17:47:04.json
../data/change_agent_game/data_sf/A1PRZC0

In [5]:
# Read all keep_close control prop data, average it and save it in a csv file
files = glob.glob("../data/change_agent_game_harder/keep_close_control_prop/*.json")

all_data = {i: [] for i in range(0, 54)}
for i, file in enumerate(files):  # Get the data for each participant in that level
    data = json.load(open(file))

    for lvl in data.keys():
        all_data[int(lvl)].append(data[lvl])

# Average the data
avg_data = {lvl: statistics.mean(all_data[lvl]) for lvl in all_data.keys()}

# Save the data in a csv file
df = pd.DataFrame({'level': avg_data.keys(), 'keep_close_control_prop': avg_data.values()})
df.to_csv('../stats/keep_close_control_prop.csv')